In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
from glob import glob

data_path = "C:/Users/emily/Documents/training/training_setA"
file_list = glob(os.path.join(data_path, "*.psv"))

df_list = [
    pd.read_csv(file, sep="|").assign(Patient_ID=i)  
    for i, file in enumerate(file_list)
]
hospitalA_df = pd.concat(df_list, ignore_index=True)
hospitalA_df = hospitalA_df.sort_values(by=["Patient_ID", "ICULOS"]).reset_index(drop=True)


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

df = hospitalA_df.copy() 
target = "SepsisLabel"
features = [col for col in df.columns if col != target]  # Use all columns except target

df[features] = df.groupby("Patient_ID")[features].fillna(method='ffill').fillna(method='bfill')
df = df.dropna(subset=[target])

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC-ROC Score: {auc:.4f}")
#precision
#recall


In [ ]:
# Plot feature importance
xgb.plot_importance(model, importance_type="weight", max_num_features=36)  # Show top 10 features
plt.show()